# Домашнее задание 2 


Продолжим обработку данных с Твиттера. 

1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
Исключим стоп-слова с помощью stop_words='english'.
Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().


2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
Исключим стоп-слова с помощью stop_words='english'.
Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().



3. Натренируем gensim.models.Word2Vec модель на наших данных.
Тренировать будем на токенизированных твитах combine_df['tweet_token']
Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.



4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".



5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".



6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.


Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))и поделить финальный вектор на количество слов в твите. На выходе должен получиться wordvec_df.shape = (49159, 200).

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [2]:
import html 
from html.parser import HTMLParser

In [3]:
path = 'C:/Users/user/Documents/mydocs/nlp_geek_brains/data/'

In [4]:
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz

In [5]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

In [6]:
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

In [7]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

In [8]:
import pickle

combine_df.to_pickle("./df_1.pkl")

In [9]:
with open('df_1.pkl', 'rb') as f:
    df= pickle.load(f)

In [10]:
df.head()

,id,label,tweet,tweet_clean,tweet_token,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love yoyou take with yoyou all the time ...,"[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


## 1

Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно. 

- Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df. 
- Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000. 
- Исключим стоп-слова с помощью stop_words='english'. 
- Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().

## stemmed 

In [11]:
allwords_s=[]
for i in df.tweet_stemmed.values:
    allwords_s.extend(i)

In [12]:
len(allwords_s)

372198

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer_s = CountVectorizer(max_features=1000, max_df=0.9, stop_words='english')

# Создаем the Bag-of-Words модель
bag_of_words_s = count_vectorizer_s.fit_transform(allwords_s)

# Отобразим Bag-of-Words модель как DataFrame
feature_names_s = count_vectorizer_s.get_feature_names()

pd.DataFrame(bag_of_words_s.toarray(), columns = feature_names_s)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yoga,york,young,youtub,yoyou,yoyour,yoyoyou,yr,yrs,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372193,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
372194,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
372195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
372196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## lemmatized

In [14]:
allwords_l=[]
for i in df.tweet_lemmatized.values:
    allwords_l.extend(i)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer_l = CountVectorizer(max_features=1000, max_df=0.9, stop_words='english')

# Создаем the Bag-of-Words модель
bag_of_words_l = count_vectorizer_l.fit_transform(allwords_l)

# Отобразим Bag-of-Words модель как DataFrame
feature_names_l = count_vectorizer_l.get_feature_names()

pd.DataFrame(bag_of_words_l.toarray(), columns = feature_names_l)

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,yo,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372193,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
372194,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
372195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
372196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2

Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.


- Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
- Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
- Исключим стоп-слова с помощью stop_words='english'.
- Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().

## stemmed 

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
tfidf_vectorizer_s = TfidfVectorizer(max_features=1000, max_df=0.9, stop_words='english')
values_s = tfidf_vectorizer_s.fit_transform(allwords_s)

# Show the Model as a pandas DataFrame
feature_names_s = tfidf_vectorizer_s.get_feature_names()
pd.DataFrame(values_s.toarray(), columns = feature_names_s)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yoga,york,young,youtub,yoyou,yoyour,yoyoyou,yr,yrs,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
372194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
372195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
372196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## lemmatized

In [17]:
tfidf_vectorizer_l = TfidfVectorizer(max_features=1000, max_df=0.9, stop_words='english')
values_l = tfidf_vectorizer_l.fit_transform(allwords_l)

# Show the Model as a pandas DataFrame
feature_names_l = tfidf_vectorizer_s.get_feature_names()
pd.DataFrame(values_l.toarray(), columns = feature_names_l)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yoga,york,young,youtub,yoyou,yoyour,yoyoyou,yr,yrs,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
372194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
372195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
372196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 3 

Натренируем gensim.models.Word2Vec модель на наших данных.

Тренировать будем на токенизированных твитах combine_df['tweet_token']
Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.


In [18]:
import gensim 
from gensim.models import Word2Vec

In [19]:
df.head()

,id,label,tweet,tweet_clean,tweet_token,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love yoyou take with yoyou all the time ...,"[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


In [71]:
modelW2V = Word2Vec(sentences=df.tweet_token, size=200, window=5, min_count=2, hs = 0, negative = 10, workers= 32, seed = 34 )

size=200 - размер вектора 

min_count - минимальная частота встречаемости слова 

hs - 0 - отрицательный сэмплы будут использованы

In [25]:
#как и поставили в параметре
len(modelW2V['credit'])

200

In [72]:
modelW2V.train(sentences=df.tweet_token, total_examples=len(df.tweet_token), epochs= 20)

(6540391, 7443960)

## 4 
Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".

In [29]:
modelW2V.wv.most_similar(positive = 'dinner')

[('sushi', 0.7573079466819763),
 ('lunch', 0.7552481889724731),
 ('drinks', 0.7152596712112427),
 ('iftar', 0.702704668045044),
 ('yum', 0.7013758420944214),
 ('cafe', 0.6987038850784302),
 ('desse', 0.6963579058647156),
 ('delicious', 0.6934821605682373),
 ('cocktails', 0.6913650035858154),
 ('familytime', 0.674503743648529)]

## 5 
Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова). Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

In [30]:
#посмотрим на вектор слова 
modelW2V['food']

array([-2.3432736e-01, -5.3791696e-01, -7.2861934e-01, -1.3155681e+00,
        1.6878036e-01, -9.5620012e-01,  6.3746583e-01,  7.1460921e-01,
       -5.3534627e-01, -2.2687287e+00, -5.7592517e-01, -1.9153125e+00,
        1.7618464e+00, -6.9696732e-02,  1.7052771e+00, -1.6012211e-01,
       -6.5909910e-01,  1.8807313e+00,  6.1308843e-01, -5.2471292e-01,
        1.2642795e-01,  1.8757973e+00,  1.0810159e-01, -1.9635462e+00,
       -7.9729682e-01, -1.5920634e+00, -1.3942385e-01,  1.4101938e+00,
        5.2258921e-01,  1.5341672e+00,  9.1193819e-01, -2.0863206e+00,
       -7.8831881e-01,  1.5662991e-01, -1.2409545e+00, -9.6951014e-01,
       -5.1833183e-01,  1.2535095e+00, -7.4733216e-01, -7.7745563e-01,
       -2.9191819e-01,  1.0290412e+00, -1.3733539e-01,  4.4735432e-01,
       -5.5484408e-01,  1.7854710e+00, -5.4475033e-01,  8.8515349e-02,
        7.8618240e-01,  6.7267537e-01,  1.8534590e+00, -7.3887706e-02,
        1.2987417e+00,  5.6145316e-01, -5.6694978e-01, -2.1639152e-01,
      

## 6 
Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. 

Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.

Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать: vec += model_w2v[word].reshape((1, size))и поделить финальный вектор на количество слов в твите. На выходе должен получиться wordvec_df.shape = (49159, 200).

In [73]:
modelW2V = Word2Vec(sentences=df.tweet_token, size=200, window=5, min_count=1, hs = 0, negative = 10, workers= 32, seed = 34 )

In [79]:
df.tweet_token.isnull().sum()

0

In [107]:
tweets = {}
for i, k in enumerate (df.tweet_token):
    if len(k) > 0:
        vec = 0
        for t in k: 
            b = modelW2V[t]
            vec += b
        tweet_vektor = vec/len(k)#.tolist()
        index = df['id'].iloc[i]
        tweets[index] = tweet_vektor
    else:
        pass

In [109]:
wordvec_df = pd.DataFrame.from_dict(tweets, orient='index')

In [110]:
wordvec_df.shape

(49097, 200)

Количество строк меньше, потому что некоторые твиты 0 после предварительной обработки 